# CNN baseline comparison - y collapsed - Base Geron


In [1]:
import nibabel as nib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import numpy as np
import glob
import nibabel as nib
import os
import matplotlib.pyplot as plt
import scipy.ndimage
import random
from tensorflow.keras.layers import Dropout, Dense, Reshape, Flatten, Conv3D, Conv3DTranspose, LeakyReLU, Input, Embedding, multiply, Concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization
from functools import partial
from tensorflow.keras import models, layers

In [2]:
DefaultConv3D = partial(layers.Conv3D, kernel_size=3, activation='relu', padding="SAME")

def build_3d_cnn_baseline(input_shape):
    model = models.Sequential([
        DefaultConv3D(filters=64, kernel_size=7, input_shape=input_shape),
        layers.MaxPooling3D(pool_size=2),
        DefaultConv3D(filters=128),
        DefaultConv3D(filters=128),
        layers.MaxPooling3D(pool_size=2),
        DefaultConv3D(filters=256),
        DefaultConv3D(filters=256),
        layers.MaxPooling3D(pool_size=2),
        layers.Flatten(),
        layers.Dense(units=128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(units=64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model



input_shape = (91, 91, 146, 1)
model = build_3d_cnn_baseline(input_shape)
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 91, 91, 146, 64)   22016     
                                                                 
 max_pooling3d (MaxPooling3D  (None, 45, 45, 73, 64)   0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 45, 45, 73, 128)   221312    
                                                                 
 conv3d_2 (Conv3D)           (None, 45, 45, 73, 128)   442496    
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 22, 22, 36, 128)  0         
 3D)                                                             
                                                                 
 conv3d_3 (Conv3D)           (None, 22, 22, 36, 256)   8

In [3]:
full_schizophrenia_ids = [
    'A00009280', 'A00028806', 'A00023132', 'A00014804', 'A00016859', 'A00021598', 'A00001181', 'A00023158',
    'A00024568', 'A00028405', 'A00001251', 'A00000456', 'A00015648', 'A00002405', 'A00027391', 'A00016720',
    'A00018434', 'A00016197', 'A00027119', 'A00006754', 'A00009656', 'A00038441', 'A00012767', 'A00034273',
    'A00028404', 'A00035485', 'A00024684', 'A00018979', 'A00027537', 'A00004507', 'A00001452', 'A00023246',
    'A00027410', 'A00014719', 'A00024510', 'A00000368', 'A00019293', 'A00014830', 'A00015201', 'A00018403',
    'A00037854', 'A00024198', 'A00001243', 'A00014590', 'A00002337', 'A00024953', 'A00037224', 'A00027616',
    'A00001856', 'A00037619', 'A00024228', 'A00038624', 'A00037034', 'A00037649', 'A00022500', 'A00013216',
    'A00020787', 'A00028410', 'A00002480', 'A00028303', 'A00020602', 'A00024959', 'A00018598', 'A00014636',
    'A00019349', 'A00017147', 'A00023590', 'A00023750', 'A00031597', 'A00015518', 'A00018317', 'A00016723',
    'A00021591', 'A00023243', 'A00017943', 'A00023366', 'A00014607', 'A00020414', 'A00035003', 'A00028805',
    'A00029486', 'A00000541', 'A00028408', 'A00000909', 'A00031186', 'A00000838' ]

# schizohrenia_id that satisfy t>90, 59 in total
met_requirement_schizophrenia_ids = [
    'A00000368', 'A00000456', 'A00000541', 'A00000838', 'A00001251', 'A00001452', 'A00004507',
    'A00006754', 'A00009280', 'A00012767', 'A00013216', 'A00014607', 'A00014719', 'A00014804',
    'A00014830', 'A00015201', 'A00015648', 'A00016197', 'A00016720', 'A00016723', 'A00017147',
    'A00018317', 'A00018403', 'A00018434', 'A00018979', 'A00019293', 'A00020414', 'A00020602', 
    'A00020787', 'A00021591', 'A00021598', 'A00023158', 'A00023246', 'A00023590', 'A00023750', 
    'A00024198', 'A00024228', 'A00024568', 'A00024684', 'A00024953', 'A00024959', 'A00027410', 
    'A00027537', 'A00028303', 'A00028404', 'A00028408', 'A00028805', 'A00028806', 'A00031186', 
    'A00031597', 'A00034273', 'A00035003', 'A00035485', 'A00037034', 'A00037224', 'A00037619', 
    'A00037649', 'A00038441', 'A00038624']

full_control_ids = [
    'A00007409', 'A00013140', 'A00021145', 'A00036049', 'A00022810', 'A00002198', 'A00020895', 'A00004667',
    'A00015826', 'A00023120', 'A00022837', 'A00010684', 'A00009946', 'A00037318', 'A00033214', 'A00022490',
    'A00023848', 'A00029452', 'A00037564', 'A00036555', 'A00023095', 'A00022729', 'A00024955', 'A00024160',
    'A00011725', 'A00027487', 'A00024446', 'A00014898', 'A00015759', 'A00028409', 'A00017294', 'A00014522',
    'A00012995', 'A00031764', 'A00025969', 'A00033147', 'A00018553', 'A00023143', 'A00036916', 'A00028052',
    'A00023337', 'A00023730', 'A00020805', 'A00020984', 'A00000300', 'A00010150', 'A00024932', 'A00035537',
    'A00022509', 'A00028406', 'A00004087', 'A00035751', 'A00023800', 'A00027787', 'A00022687', 'A00023866',
    'A00021085', 'A00022619', 'A00036897', 'A00019888', 'A00021058', 'A00022835', 'A00037495', 'A00026945',
    'A00018716', 'A00026907', 'A00023330', 'A00016199', 'A00037238', 'A00023131', 'A00014120', 'A00021072',
    'A00037665', 'A00022400', 'A00003150', 'A00024372', 'A00021081', 'A00022592', 'A00022653', 'A00013816',
    'A00014839', 'A00031478', 'A00014225', 'A00013363', 'A00037007', 'A00020968', 'A00024301', 'A00024820',
    'A00035469', 'A00029226', 'A00022915', 'A00022773', 'A00024663', 'A00036844', 'A00009207', 'A00024535',
    'A00022727', 'A00011265', 'A00024546'
]

 # 82 controls that met requirement
met_requirement_control_ids = [
    'A00000300', 'A00002198', 'A00003150', 'A00004087', 'A00007409', 'A00010684', 'A00011265', 'A00011725',
    'A00012995', 'A00013140', 'A00013816', 'A00014839', 'A00014898', 'A00015759', 'A00015826', 'A00018553',
    'A00018716', 'A00019888', 'A00020805', 'A00020895', 'A00020968', 'A00020984', 'A00021058', 'A00021072',
    'A00021081', 'A00021085', 'A00022400', 'A00022490', 'A00022509', 'A00022592', 'A00022619', 'A00022653',
    'A00022687', 'A00022727', 'A00022729', 'A00022773', 'A00022810', 'A00022835', 'A00022837', 'A00022915',
    'A00023095', 'A00023120', 'A00023131', 'A00023143', 'A00023330', 'A00023337', 'A00023730', 'A00023800',
    'A00023848', 'A00023866', 'A00024160', 'A00024301', 'A00024372', 'A00024446', 'A00024535', 'A00024546', 
    'A00024663', 'A00024820', 'A00024932', 'A00024955', 'A00025969', 'A00026945', 'A00027487', 'A00027787', 
    'A00028052', 'A00028406', 'A00028409', 'A00029226', 'A00029452', 'A00031478', 'A00031764', 'A00033214', 
    'A00035751', 'A00036049', 'A00036555', 'A00036844', 'A00037007', 'A00037238', 'A00037318', 'A00037495', 
    'A00037564', 'A00037665'
]

In [4]:
# GAN Training Data Selection
gan_train_ids_schiz = random.sample(met_requirement_schizophrenia_ids, 4)
gan_test_ids_schiz = [id for id in met_requirement_schizophrenia_ids if id not in gan_train_ids_schiz]

gan_train_ids_control = random.sample(met_requirement_control_ids, 4)
gan_test_ids_control = [id for id in met_requirement_control_ids if id not in gan_train_ids_control]
gan_test_ids_control = random.sample(gan_test_ids_control,4)

''' data training for classifier '''
''' just use the same train set as GAN above '''

# Classifier Training Data Selection
#classifier_train_ids = gan_train_ids_control + gan_train_ids_schiz

# Classifier Test Data Selection
classifier_test_ids = gan_test_ids_schiz + gan_test_ids_control

''' File loading '''
# Specify the directory and file pattern
directory_path = '4D/'
file_pattern = 'A*_????_func_FL_FD_RPI_DSP_MCF_SS_SM_Nui_CS_InStandard.nii.gz'

# Construct the full path pattern
path_pattern = f'{directory_path}/{file_pattern}'

# Use glob to find all matching files
matching_files = glob.glob(path_pattern)

''' File loading for GAN Training and classifer '''
''' But this time we have 2 separate GANs, 1 train on schizoprenia and 1 train on control'''

#classifier_image_data = []
#classifier_labels = []  # 1 for schizophrenia, 0 for non-schizophrenia
gan_image_data_schiz = []
gan_image_data_control = []

for file_path in matching_files:
    filename = os.path.basename(file_path)
    file_id = filename.split('_')[0]
    
    if file_id in gan_train_ids_schiz:
        t1_img = nib.load(file_path)
        t1_data = t1_img.get_fdata()

        if t1_data.shape[3] < 90:
            continue

        t1_data_collapsed = np.sum(t1_data, axis=1)
        gan_image_data_schiz.append(t1_data_collapsed)

    if file_id in gan_train_ids_control:
        t1_img = nib.load(file_path)
        t1_data = t1_img.get_fdata()

        if t1_data.shape[3] < 90:
            continue

        t1_data_collapsed = np.sum(t1_data, axis=1)
        gan_image_data_control.append(t1_data_collapsed)

    # if file_id in classifier_train_ids or file_id in classifier_test_ids:
    #     t1_img = nib.load(file_path)
    #     t1_data = t1_img.get_fdata()

    #     if t1_data.shape[3] < 90:
    #         continue

    #     label = 1 if file_id in met_requirement_schizophrenia_ids else 0
    #     t1_data_collapsed = np.sum(t1_data, axis=1)

    #     classifier_image_data.append(t1_data_collapsed)
    #     classifier_labels.append(label)

print(f"Total GAN control loaded: {len(gan_image_data_control)}")
print(f"Total GAN schiz loaded: {len(gan_image_data_schiz)}")
#print(f"Total classifier training/testing files processed: {len(classifier_image_data)}")
#print(f"Total labels processed: {len(classifier_labels)}")


'''Determine the maximum 3rd-dimension size '''
max_z_size_schiz = max(img.shape[2] for img in gan_image_data_schiz)
max_z_size_control = max(img.shape[2] for img in gan_image_data_control)
#max_z_size_classifier = max(img.shape[2] for img in classifier_image_data)
max_z_size = max(max_z_size_schiz,max_z_size_control)

''' normalization '''
image_data_normalized_schiz = [(img - np.min(img)) / (np.max(img) - np.min(img)) * 2 - 1 for img in gan_image_data_schiz]
image_data_normalized_control = [(img - np.min(img)) / (np.max(img) - np.min(img)) * 2 - 1 for img in gan_image_data_control]
#image_data_normalized_classifier = [(img - np.min(img)) / (np.max(img) - np.min(img)) * 2 - 1 for img in classifier_image_data]

''' padding of images data '''
# Pad each image to have a consistent z-dimension size
padded_data_schiz = [np.pad(img, ((0, 0), (0, 0), (0, max_z_size - img.shape[2])), mode='constant') for img in image_data_normalized_schiz]
padded_data_control = [np.pad(img, ((0, 0), (0, 0), (0, max_z_size - img.shape[2])), mode='constant') for img in image_data_normalized_control]
#padded_data_classifier = [np.pad(img, ((0, 0), (0, 0), (0, max_z_size_classifier - img.shape[2])), mode='constant') for img in image_data_normalized_classifier]

# Now convert the padded data list to a numpy array
padded_data_array_schiz = np.array(padded_data_schiz)
padded_data_array_control = np.array(padded_data_control)
#padded_data_array_classifier = np.array(padded_data_classifier)

''' loading the data for WGAN training '''
train_images_schiz = padded_data_array_schiz
train_images_control = padded_data_array_control

# Define batch size
batch_size = 10


train_dataset_schiz = tf.data.Dataset.from_tensor_slices((train_images_schiz)).shuffle(len(train_images_schiz)).batch(batch_size)
train_dataset_control = tf.data.Dataset.from_tensor_slices((train_images_control)).shuffle(len(train_images_control)).batch(batch_size)

''' setting up parameters'''
# Image shape and other parameters
#img_shape = (84, 84, 72, 1)
#z_dim = 100

Total GAN control loaded: 4
Total GAN schiz loaded: 4


' setting up parameters'

In [5]:
print(max_z_size_control)

145


In [6]:
print(max_z_size_schiz)

146


In [7]:
# Create labels for the datasets
labels_schiz = np.ones(len(padded_data_array_schiz))  # 1 for schizophrenia
labels_control = np.zeros(len(padded_data_array_control))  # 0 for control

# Combine the data and labels
combined_images = np.concatenate([padded_data_array_schiz, padded_data_array_control], axis=0)
combined_labels = np.concatenate([labels_schiz, labels_control], axis=0)

# Shuffle the combined dataset to ensure the data is randomly distributed
# It's important for training a model that generalizes well
indices = np.arange(combined_images.shape[0])
np.random.shuffle(indices)
shuffled_images = combined_images[indices]
shuffled_labels = combined_labels[indices]

# Convert to TensorFlow Dataset, manually split 80/20
dataset = tf.data.Dataset.from_tensor_slices((shuffled_images, shuffled_labels))
train_size = int(0.8 * len(shuffled_images))
train_dataset = dataset.take(train_size).shuffle(buffer_size=train_size).batch(10)
test_dataset = dataset.skip(train_size).batch(10)

# Model initialization
#input_shape = shuffled_images.shape[1:]  # This includes the channel dimension if already added
#print(input_shape)
#input_shape = (91, 91, 146, 1)
#model = build_3d_cnn(input_shape=input_shape, num_classes=1)

# Train the model
history = model.fit(
    train_dataset,
    epochs=200,
    validation_data=test_dataset
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc}, Test loss: {test_loss}")


Epoch 1/1000


ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/sequential/max_pooling3d_1/MaxPool3D/MaxPool3DGrad' defined at (most recent call last):
    File "c:\Users\josep\anaconda3\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\josep\anaconda3\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\josep\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\josep\anaconda3\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\josep\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\josep\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\josep\anaconda3\lib\asyncio\base_events.py", line 

In [ ]:
def resize_image(image, new_shape):
    factors = (
        new_shape[0]/image.shape[0],
        new_shape[1]/image.shape[1],
        new_shape[2]/image.shape[2]
    )
    return scipy.ndimage.zoom(image, factors, order=1)  # order=1 is bilinear interpolation

''' load test images to make test set'''
# Now proceed with loading and preprocessing the images for these IDs
test_image_data = []
test_labels = []

test_ids = classifier_test_ids

# Loop through the matching files and filter based on test IDs
for file_path in matching_files:
    filename = os.path.basename(file_path)
    file_id = filename.split('_')[0]

    # Process only if the ID is in the test set
    if file_id in test_ids:
        t1_img = nib.load(file_path)
        t1_data = t1_img.get_fdata()

        # Ensure sufficient time dimension
        if t1_data.shape[3] < 90:
            continue

        # Collapse one of the axes by summing
        t1_data_collapsed = np.sum(t1_data, axis=1)

        # Resize, normalize, and add dimension as done in the training data preparation
        processed_image = resize_image(t1_data_collapsed, (91, 91, 146))
        processed_image_normalized = (processed_image - np.min(processed_image)) / (np.max(processed_image) - np.min(processed_image)) * 2 - 1
        processed_image_final = np.expand_dims(processed_image_normalized, axis=-1)

        test_image_data.append(processed_image_final)
        label = 1 if file_id in met_requirement_schizophrenia_ids else 0
        test_labels.append(label)

# Convert to numpy arrays
test_images_array = np.array(test_image_data)
test_labels_array = np.array(test_labels)


test_dataset = tf.data.Dataset.from_tensor_slices((test_images_array, test_labels_array)).batch(batch_size)

loss, accuracy = model.evaluate(test_dataset)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)